# DEM Download, Co-Registration, and Differencing for Geodetic Glacier Mass Balance Analysis 

### This Python workflow will walk you through the steps to: 

1. Pull the SRTM DEM corresponding to your study area from EarthAccess -OR- load an existing reference DEM of your choosing 
2. Load the additional DEM(s) you wish to co-register and the glacier polygons of your choosing (RGI, GLIMS, DGA, etc.)  
3. Perform DEM co-registration, following the workflow of Shean et al. (2023), an implementation of Nuth and Kääb (2011)
4. Clip the co-registered DEMs to glacier area 
5. Difference the DEMs to compute volume loss within glacier area 
6. Calculate geodetic glacier mass balance 
7. Produce figures  

This workflow has the following requirements: 
- a NASA EarthData login to download the SRTM elevation data. EarthData can be accessed at the following link: (https://urs.earthdata.nasa.gov/oauth/authorize?client_id=ZAQpxSrQNpk342OR77kisA&response_type=code&redirect_uri=https://appeears.earthdatacloud.nasa.gov/login&state=/)
- additional DEMs of the study area (e.g. UAV or satellite-derived product)

We use `earthaccess`, an open source package developed by Luis Lopez to download NASA data collections. It requires authentication using Earthdata Login credentials. See [https://github.com/nsidc/earthaccess](https://github.com/nsidc/earthaccess).

MS note to self: 
to activate virtual environment locally (in terminal): 

cd ~/Glacier-DEM-coregistration-and-MB
source .venv/bin/activate

In [1]:
# # Install dependencies 
# import sys
# !{sys.executable} -m pip install earthaccess
# !pip install geopandas
%pip install pyproj

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 1: Pull SRTM for study area as reference DEM 

In [5]:
#Code to download SRTM DEM from EarthAccess

# For searching and accessing NASA data
import earthaccess
import os

# Enter your EarthAccess login and password 
earthaccess.login()

# search for SRTM data
query = earthaccess.search_datasets(
            keyword="SRTM",
)

bounding_box = (-71.4, -36.8, -71.2, -36.6) #update the bounding box for your study area 
#  bbox is defined as [min_lon, min_lat, max_lon, max_lat]

results = earthaccess.search_data(
    short_name = 'SRTMGL1', # SRTM 1-arcsecond Global dataset (confirm this is the right SRTM dataset...)
    cloud_hosted = True, # Ensure access to cloud-stored data
    bounding_box=bounding_box
)

# Define output directory
output_dir = "./srtm_data"
os.makedirs(output_dir, exist_ok=True)

# Download the files
for granule in results:
    earthaccess.download(granule, output_dir)

## add step / print statement to confirm download and confirm that the bounding box is the correct study area?? 
print(f"Downloaded {len(results)} SRTM DEM files to: {os.path.abspath(output_dir)}")

# Print study area bounding box
print(f"Study area bounding box: {bounding_box}")  


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 2839.75it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 14463.12it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 39945.75it/s]

Downloaded 1 SRTM DEM files to: /Users/milliespencer/Glacier-DEM-coregistration-and-MB/srtm_data
Study area bounding box: (-71.4, -36.8, -71.2, -36.6)


## Step 2a: Load the glacier shapefiles and the additional DEMs you wish to work with 

Below is the code to unzip Chilean glacier inventory shapefiles (DGA,2000 or DGA,2019) OR use unzipped example shapefiles for Nevados de Chillán complex. If your study area is outside of Chile, you can download glacier polygons for your study area from GLIMS here: https://www.glims.org/maps/glims and manually upload below. 

In [ ]:
from pyproj import CRS
crs = CRS("EPSG:32719")
print(crs)

## not sure why i can't get this to run - any thoughts? 

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)

In [4]:
# comment out code that doesn't apply depending on which glacier polygon source you choose
# note for Emma, I need to make a new shapefile with just the polygons over Nevados de Chillán, I can do this next week! Required for the first chunk of code below to run
import geopandas as gpd
from pyproj import CRS

# Define a CRS using an EPSG code
crs = CRS.from_epsg(32719)
print(crs)

# # if using example shapefiles (unzipped) for Nevados de Chillán 
glacier_area_2000_path = "example_data_Nevados/Nevados_glacier_shapefiles/Nevados_shapefile_DGA2000/Nevados_polygons_DGA2000.shp"
glacier_area_2019_path = "example_data_Nevados/Nevados_glacier_shapefiles/Nevados_shapefile_DGA2019/Nevados_polygons_DGA2019.shp"

# Load the shapefile
glacier_2000_gdf = gpd.read_file(glacier_area_2000_path)
glacier_2019_gdf = gpd.read_file(glacier_area_2019_path)

# Display the first few rows
print(glacier_2000_gdf.head())
print(glacier_2019_gdf.head())


# ## if using full zipped DGA year 2000 or 2019 glacier inventory, below is example to unzip year 2000 DGA inventory 
# import zipfile

CRSError: Invalid projection: EPSG:32719: (Internal Proj Error: proj_create: no database context specified)

## Step 2b: Load the additional DEMs you wish to work with 

As an example, we provide an SRTM DEM, a DEM produced by 1954 Chilean military flights (IGM_1954), and two DEMs derived from uncrewed aerial vehicle (UAV) flights over Nevados de Chillán in March, 2024 (CerroBlanco and LasTermas). You can manually upload your DEMs to the same folder or route to a local file on your computer. Alternatively, you can modify the code in Step 1 to pull other DEM products from online platforms (e.g. IceSAT/-2, ArcticDEM, etc.). 

We also provide the GLIMS polygons in raster format ... 

In [ ]:
## code to call all DEMs at once 
import rasterio
import os

# Path to your data folder
DEM_folder = "/Users/milliespencer/Glacier-DEM-coregistration-and-MB/example_data_Nevados/Nevados_DEMs"

# List of DEM filenames
dem_files = [
    "CerroBlanco_2024.tif",
    "LasTermas_2024.tif",
    "SRTM_2000.tif",
    "IGM_1954.tif"
]

# Dictionary to hold opened DEM datasets
dem_datasets = {}

# Loop to open all DEMs
for dem_file in dem_files:
    file_path = os.path.join(DEM_folder, dem_file)
    dem_name = os.path.splitext(dem_file)[0]  # Remove .tif extension
    dem_datasets[dem_name] = rasterio.open(file_path)
    # dem_datasets[dem_name] = rasterio.open(file_path).read(1) 
    # #If you plan to read them into arrays immediately, you can add .read(1) inside the loop^


# Optional: print CRS of each DEM to confirm they’re loaded
for name, dataset in dem_datasets.items():
    print(f"{name}: CRS = {dataset.crs}, Resolution = {dataset.res}")

# You can later access each DEM like this: 
dem_datasets["SRTM_2000"]

## Step 3: Perfom DEM co-registration 
Here we follow the workflow provided by Shean et al. (2023), derived from the Nuth and Kääb (2011) method, here: https://github.com/dshean/demcoreg/blob/master/docs/beginners_doc.md
NB: users should confirm that the co-registration workflow has not been modified since the date of this code publication. 

### Co-registration setup: 
Loading the DEMs, reprojecting the DEMs to the same spatial reference system, extent, and cell size, and calculating slope and aspect to use in the co-registration process:

In [ ]:
## issue... I did all this in the command line so idk how to translate into a python notebook... 